In [1]:
%load_ext autoreload
%autoreload 2

import os
import os.path as osp

from nscl.datasets.clevr_dataset import build_clevr_dataset, build_clevr_dataloader, CLEVRCurriculumSampler
from nscl.datasets.clevr_definition import CLEVRDefinition
from nscl.models.nscl_module import NSCLModule

In [2]:
train_img_root = osp.abspath(osp.dirname(os.getcwd())) + '/data/CLEVR_v1.0/images/train'
train_scene_json = osp.abspath(osp.dirname(os.getcwd())) + '/data/CLEVR_v1.0/scenes/train/scenes.json'
train_question_json = osp.abspath(osp.dirname(os.getcwd())) + '/data/CLEVR_v1.0/questions/CLEVR_train_questions.json'

In [3]:
batch_size = 100
num_workers = 4
dataset = build_clevr_dataset(train_img_root, train_scene_json, train_question_json)

loading scenes from: /Users/mark/Projects/nscl_reproducability_challenge/data/CLEVR_v1.0/scenes/train/scenes.json
loading questions from: /Users/mark/Projects/nscl_reproducability_challenge/data/CLEVR_v1.0/questions/CLEVR_train_questions.json


In [4]:
curriculum_sampler = CLEVRCurriculumSampler(dataset, max_scene_size=5, max_program_size=5, max_data_size=1000)
data_loader = build_clevr_dataloader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False,
                                     drop_last=False, sampler=curriculum_sampler)

Preparing curriculum sampler....


In [6]:
import torch
from torch import nn
from torch import optim

epoch = 10
model = NSCLModule(CLEVRDefinition.attribute_concept_map).float()
criterion = nn.MSELoss()# nn.KLDivLoss(reduction='batchmean')
opt = optim.Adam(model.parameters())


for e in range(epoch):
    model.eval()
    for images, questions, scenes in data_loader:
        print('running...')
        opt.zero_grad()
        results = model(images, questions, scenes)
        total_loss = torch.tensor(0.0, requires_grad=True, dtype=torch.float)
        for i, q in enumerate(questions):
            print(q.raw_question, q.answer)
            print(q.answer_tensor)
            print(results[i])
            loss = criterion(results[i], q.answer_tensor)
            total_loss = total_loss + loss
            print()
        total_loss.backward()
        opt.step()
        print(f'Epoch {e} loss:', total_loss)


running...
The purple rubber thing has what shape? cube
tensor([1., 0., 0.])
tensor([0.0196, 0.8563, 0.1241], grad_fn=<DivBackward0>)

The tiny cube has what color? green
tensor([0., 0., 0., 1., 0., 0., 0., 0.])
tensor([0.1821, 0.0188, 0.1529, 0.1760, 0.0061, 0.0695, 0.0471, 0.3475],
       grad_fn=<DivBackward0>)

Is there a big matte ball? yes
tensor(1.)
tensor(0.0335, grad_fn=<MaxBackward1>)

What is the small yellow thing made of? rubber
tensor([1., 0.])
tensor([0.9456, 0.0544], grad_fn=<DivBackward0>)

What is the size of the blue block? large
tensor([0., 1.])
tensor([0.7832, 0.2168], grad_fn=<DivBackward0>)

What is the green cylinder made of? metal
tensor([0., 1.])
tensor([0.3844, 0.6156], grad_fn=<DivBackward0>)

What is the size of the gray matte object? large
tensor([0., 1.])
tensor([0.2313, 0.7687], grad_fn=<DivBackward0>)

There is a small yellow object; what shape is it? sphere
tensor([0., 1., 0.])
tensor([0.0142, 0.8882, 0.0976], grad_fn=<DivBackward0>)

Are there any big